> **Note:** In most sessions you will be solving exercises posed in a Jupyter notebook that looks like this one. Because you are cloning a Github repository that only we can push to, you should **NEVER EDIT** any of the files you pull from Github. Instead, what you should do, is either make a new notebook and write your solutions in there, or **make a copy of this notebook and save it somewhere else** on your computer, not inside the `sds` folder that you cloned, so you can write your answers in there. If you edit the notebook you pulled from Github, those edits (possible your solutions to the exercises) may be overwritten and lost the next time you pull from Github. This is important, so don't hesitate to ask if it is unclear.

In [3]:
import scraping_class
logfile = 'log.txt' ## name your log file.
connector = scraping_class.Connector(logfile)

# Exercise Set 7: Parsing and Information Extraction

In this Exercise Set we shall develop our webscraping skills even further by practicing **parsing** and navigating html trees using BeautifoulSoup and furthermore train extracting information from raw text with no html tags to help, using regular expressions. 

But just as importantly you will get a chance to think about **data quality issues** and how to ensure reliability when curating your own webdata. 

## Exercise Section 7.1: Logging and data quality

> **Ex. 7.1.1:** *Why is is it important to log processes in your data collection?*



In [2]:
# [Answer to Ex. 7.1.1 here]

When designing your own data collection you are in charge of ensuring the quality. Many processes can go wrong, which if not spotted can lead to fundamental distortions of your dataset and in turn conclusions. 

> **Ex. 7.1.2:**
*How does logging help with both ensuring and documenting the quality of your data?*


In [2]:
# [Answer to Ex. 7.1.2 here]

Having a log allow you to spot the potential anomalies in the website you are scraping. Examples could be errors and failed calls not randomly distributed accross time or subdomains. This would prompt you to investigate the issue further. Inspecting the simple distribution of lenght of the response, can also tell you about potential artifacts in your data collection that you should look into. Maybe there are more than one template important for your parsing, or there is a certain standard response given when data is missing.
When presenting simple statistics about the data collection, it provides transparency and signals the commitment you have made as a data curator.

## Exercise Section 7.2: Parsing a Table from HTML using BeautifulSoup.

Yesterday I showed you a neat little prepackaged function in pandas that did all the work. However today we should learn the mechanics of it. *(It is not just for educational purposes, sometimes the package will not do exactly as you want.)*

We hit the Basketball stats page from yesterday again: https://www.basketball-reference.com/leagues/NBA_2018.html.


> **Ex. 7.2.1:** Here we practice simply locating the table node of interest using the `find` method build into BeautifoulSoup. But first we have to fetch the HTML using the `requests` module. Parse the tree using `BeautifulSoup`. And then use the **>Inspector<** tool (* right click on the table < press inspect element *) in your browser to see how to locate the Eastern Conference table node - i.e. the *tag* name of the node, and maybe some defining *attributes*.

In [1]:
# [Answer to Ex. 7.2.1 here]

In [4]:
from bs4 import BeautifulSoup
import requests
url = 'https://www.basketball-reference.com/leagues/NBA_2018.html'
response = connector.get(url,'table_exercise')
html = response[0].text
soup = BeautifulSoup(html,'html.parser')
table_node = soup.find('table',attrs={'id':'confs_standings_E'}) # find table node with specific attribute

You have located the table should now build a function that starts at a "table node" and parses the information, and outputs a pandas DataFrame. 

Inspect the element either within the notebook or through the **>Inspector<** tool and start to see how a table is written in html. Which tag names can be used to locate rows? How will you iterate through columns. Were is the header located?

> **Ex. 7.1.2:** First you parse the header which can be found in the canonical tag name: thead. 
Next you use the `find_all` method to search for the tag, and iterate through each of the elements extracting the text, using the `.text` method builtin to the the node object. Store the header values in a list container. 

> **Ex. 7.1.3:** Next you locate the rows, using the canonical tag name: tbody. And from here you search for all rows tags. Fiugre out the tag name yourself, inspecting the tbody node in python or using the **Inspector**. 

> **Ex. 7.1.4:** Next run through all the rows and extract each value, similar to how you extracted the header. However here is a slight variation: Since each value node can have a different tag depending on whether it is a digit or a string, you should use the `.children` method instead of the `.find_all` - (or write compile a regex that matches both the td tag and the th tag.) 
>Once the value nodes of each row has been located using the `.children` method you should extract the value. Store the extracted rows as a list of lists: ```[[val1,val2,...valk],...]```

In [1]:
# [Answer to Ex. 7.2.2-4 here]

In [5]:
import pandas as pd
name = table_node.caption.text

# parse header
header = table_node.thead.find_all('th') # locate each column name using the `th` tag, which entail a string .

# extract the label you want. brevity use .text, for a more informative get teh aria-label attribute
header = [col['aria-label'] for col in header]

print(header,len(header))

# parse rows: the canonical tbody locates the data body.
body = table_node.tbody

# rows are found using the "tr" tag
rows = body.find_all('tr')

# each row value has different tags depending on their type (digit or string)
# function to check what tag it is and either convert to float or not. 
import numpy as np
def convert_value_type(value_node):
    if value_node.name == 'th':
        return value_node.text
    else: # assume node is td:
        try: 
            return float(value_node.text)
        except:
            return np.nan # assume there is no value if it fails. 
data = []
for row_node in rows:
    row = []
    for child in row_node.children:
        row.append(convert_value_type(child))
    data.append(row)


['Eastern Conference', 'Wins', 'Losses', 'Win-Loss Percentage', 'Games Behind', 'Points Per Game', 'Opponent Points Per Game', 'Simple Rating System'] 8


In [6]:

df = pd.DataFrame(data,columns=header)
def parse_html_table(table_node):
    # parse header
    header = table_node.thead.find_all('th') # locate each column name using the `th` tag, which entail a string .
    # extract the label you want. brevity use .text, for a more informative get teh aria-label attribute
    header = [col['aria-label'] for col in header]
    # parse rows: the canonical tbody locates the data body.
    body = table_node.tbody
    # rows are found using the "tr" tag
    rows = body.find_all('tr')
    # each row value has different tags depending on their type (digit or string)
    # function to check what tag it is and either convert to float or not. 
    import numpy as np
    def convert_value_type(value_node):
        if value_node.name == 'th':
            return value_node.text
        else: # assume node is td:
            try: 
                return float(value_node.text)
            except:
                return np.nan # assume there is no value if it fails. 
    data = []
    for row_node in rows:
        row = []
        for child in row_node.children:
            row.append(convert_value_type(child))
        data.append(row)
    df = pd.DataFrame(data,columns=header)
    return df
df = parse_html_table(table_node)
df.head()

,Eastern Conference,Wins,Losses,Win-Loss Percentage,Games Behind,Points Per Game,Opponent Points Per Game,Simple Rating System
0,Toronto Raptors*,59.0,23.0,0.720,NaN,111.7,103.9,7.29
1,Boston Celtics*,55.0,27.0,0.671,4.0,104.0,100.4,3.23
2,Philadelphia 76ers*,52.0,30.0,0.634,7.0,109.8,105.3,4.30
3,Cleveland Cavaliers*,50.0,32.0,0.610,9.0,110.9,109.9,0.59
4,Indiana Pacers*,48.0,34.0,0.585,11.0,105.6,104.2,1.18


> **Ex. 7.2.6:** Now locate all tables from the page, using the `.find_all` method searching for the table tag name. Iterate through the table nodes and apply the function created for parsing html tables. Store each table in a dictionary using the table name as key. The name is found by accessing the id attribute of each table node, using dictionary-style syntax - i.e. `table_node['id']`.

In [1]:
# [Answer to Ex. 7.2.2-4 here]

In [5]:
tables = soup.find_all('table') # locate all table nodes
dfs = {}
for table_node in tables:
    name = table_node['id'] # access the id attribute. 
    table = parse_html_table(table_node) # apply parse_table function
    dfs[name] = table # store table in the dictionary

> **Ex. 7.2.7. (extra) :** Compare your results to the pandas implementation. pd.read_html